<a href="https://colab.research.google.com/github/AhmedAboulezz/Trading/blob/main/Predict_Open_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#===============================================================================
# CELL 1: Load strategy trades + prices/indicators
#===============================================================================

!pip install pandas numpy scikit-learn --quiet

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# لو حابب ترفع الملفات يدوي من جهازك:
# from google.colab import files
# uploaded = files.upload()

# غيّر دول حسب اسماء الملفات عندك:
strategy_csv_path = "StrategyOutputEdited.csv"       # ملف الاستراتيجية
prices_csv_path   = "BTC-45-Prices.csv"     # ملف الأسعار + المؤشرات


#---------------------- Load strategy file ----------------------#

# توقع أسماء الأعمدة زي المثال اللي بعتهالي:
# Trade #	Type	Date/Time	Signal	Price USDT	Position size (qty)	Position size (value)
# Net P&L USDT	Net P&L %	Run-up USDT	Run-up %	Drawdown USDT	Drawdown %
# Cumulative P&L USDT	Cumulative P&L %

raw_trades = pd.read_csv(strategy_csv_path)

print("Strategy file columns:")
print(list(raw_trades.columns))

rename_map = {
    "Trade #": "trade_id",
    "Type": "type",
    "Date/Time": "datetime",
    "Signal": "signal",
    "Price USDT": "price",
    "Position size (qty)": "pos_qty",
    "Position size (value)": "pos_value",
    "Net P&L USDT": "net_pnl_usdt",
    "Net P&L %": "net_pnl_pct",
    "Run-up USDT": "runup_usdt",
    "Run-up %": "runup_pct",
    "Drawdown USDT": "drawdown_usdt",
    "Drawdown %": "drawdown_pct",
    "Cumulative P&L USDT": "cum_pnl_usdt",
    "Cumulative P&L %": "cum_pnl_pct",
}

# نعمل rename على الموجود بس
rename_map_existing = {k: v for k, v in rename_map.items() if k in raw_trades.columns}
raw_trades = raw_trades.rename(columns=rename_map_existing)

# تأكد إن عندنا الأعمدة المهمة
required_cols = ["trade_id", "type", "datetime", "price",
                 "net_pnl_pct", "runup_pct", "drawdown_pct"]
missing = [c for c in required_cols if c not in raw_trades.columns]
if missing:
    print("⚠️ Missing in strategy file:", missing)

# parse datetime
raw_trades["datetime"] = pd.to_datetime(raw_trades["datetime"])

# نرتب حسب رقم التريد و الوقت
raw_trades = raw_trades.sort_values(["trade_id", "datetime"]).reset_index(drop=True)

print("\nSample from strategy file after rename:")
print(raw_trades.head(8))


#---------------------- Build one-row-per-trade table ----------------------#

def build_trades_table(df):
    """
    لكل trade_id:
      - نلاقي صف الدخول (Type يحتوي 'Entry')
      - نلاقي صف الخروج (Type يحتوي 'Exit') لكن نتجاهل الصفوف اللي Signal فيها 'Open'
        (لأنها بتمثل صفقة لسه مفتوحة).
    """
    trades = []
    for tid, g in df.groupby("trade_id"):
        g = g.sort_values("datetime")

        # صفوف الدخول
        entry_rows = g[g["type"].str.lower().str.contains("entry")]

        # صفوف الخروج الحقيقية بس (signal فيها close مش open)
        exit_rows = g[
            g["type"].str.lower().str.contains("exit")
            & ~g["signal"].str.lower().str.contains("open")
        ]

        if len(entry_rows) == 0:
            continue

        entry = entry_rows.iloc[-1]   # آخر entry
        exit_row = exit_rows.iloc[-1] if len(exit_rows) > 0 else None

        trades.append({
            "trade_id": tid,
            "entry_time": entry["datetime"],
            "entry_price": float(entry["price"]),
            "exit_time": exit_row["datetime"] if exit_row is not None else pd.NaT,
            "exit_price": float(exit_row["price"]) if exit_row is not None else np.nan,
            "net_pnl_pct": float(entry.get("net_pnl_pct", np.nan)),
            "runup_pct": float(entry.get("runup_pct", np.nan)),
            "drawdown_pct": float(entry.get("drawdown_pct", np.nan)),
        })

    trades = pd.DataFrame(trades).sort_values("entry_time").reset_index(drop=True)
    return trades



#---------------------- Load prices + indicators file ----------------------#

prices = pd.read_csv(prices_csv_path)

print("\nPrices/indicators file columns:")
print(list(prices.columns))

# نعمل rename للأعمدة المهمة حسب اللي بعتهالي
price_rename = {
    "time": "time",
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "VWMA": "vwma",
    "Volume": "volume",
    "RSI": "rsi",
    "RSI-based MA": "rsi_ma",
    "Regular Bullish": "reg_bull",
    "Regular Bullish Label": "reg_bull_label",
    "Regular Bearish": "reg_bear",
    "Regular Bearish Label": "reg_bear_label",
    "%K": "stoch_k",
    "%D": "stoch_d",
    "Histogram": "macd_hist",
    "MACD": "macd",
    "Signal": "macd_signal",
    "OnBalanceVolume": "obv",
    "ATR": "atr",
    "ROC": "roc",
    "MOM": "mom",
    "CCI": "cci",
    "CCI-based MA": "cci_ma",
    "Conversion Line": "conv_line",
    "Base Line": "base_line",
    "Leading Span A": "span_a",
    "Leading Span B": "span_b",
}

price_rename_existing = {k: v for k, v in price_rename.items() if k in prices.columns}
prices = prices.rename(columns=price_rename_existing)

# parse time
if "time" not in prices.columns:
    raise ValueError("Prices file must contain a 'time' column.")
prices["time"] = pd.to_datetime(prices["time"])

# نرتب
prices = prices.sort_values("time").reset_index(drop=True)

print("\nSample from prices/indicators after rename:")
print(prices.head(10))


Strategy file columns:
['Trade #', 'Type', 'Date/Time', 'Signal', 'Price USDT', 'Position size (qty)', 'Position size (value)', 'Net P&L USDT', 'Net P&L %', 'Run-up USDT', 'Run-up %', 'Drawdown USDT', 'Drawdown %', 'Cumulative P&L USDT', 'Cumulative P&L %']

Sample from strategy file after rename:
   trade_id        type            datetime                     signal  \
0         1  Entry long 2024-01-03 08:30:00                       Long   
1         1   Exit long 2024-01-04 08:30:00  Close entry(s) order Long   
2         2  Entry long 2024-01-05 13:00:00                       Long   
3         2   Exit long 2024-01-05 19:00:00  Close entry(s) order Long   
4         3  Entry long 2024-01-07 22:45:00                       Long   
5         3   Exit long 2024-01-09 11:30:00  Close entry(s) order Long   
6         4  Entry long 2024-01-12 04:00:00                       Long   
7         4   Exit long 2024-01-15 14:30:00  Close entry(s) order Long   

     price   pos_qty    pos_value 

In [23]:
#===============================================================================
# CELL 2: Join indicators at trade entry & build ML dataset
#===============================================================================

# نستخدم نفس الأسامي اللي طلعت من Cell 1:
# trades  = جدول الصفقات (من build_trades_table)
# prices  = الأسعار + المؤشرات

# نتأكد إن الاتنين متسوّرين بالوقت
trades_sorted = trades.sort_values("entry_time").copy()
prices_sorted = prices.sort_values("time").copy()

# 🟢 أهم خطوة: توحيد نوع الـ datetime (ونخليهم UTC عشان timezone -05:00 في الأسعار)
trades_sorted["entry_time"] = pd.to_datetime(trades_sorted["entry_time"], utc=True)
prices_sorted["time"]       = pd.to_datetime(prices_sorted["time"], utc=True)

# دلوقتي نقدر نعمل merge_asof عادي: ياخد آخر بار <= entry_time
merged = pd.merge_asof(
    trades_sorted.sort_values("entry_time"),
    prices_sorted.sort_values("time"),
    left_on="entry_time",
    right_on="time",
    direction="backward"
)

print("Merged trades + features sample:")
print(merged.head(10))

# نشتغل فقط على الصفقات المقفولة للتدريب
closed = merged[~merged["exit_time"].isna()].copy()
open_for_pred = merged[merged["exit_time"].isna()].copy()

print(f"\nClosed trades for training: {len(closed)}")
print(f"Open trades for prediction: {len(open_for_pred)}")
if len(open_for_pred) > 0:
    print(open_for_pred[["trade_id", "entry_time", "entry_price", "time", "close"]].tail())

#---------------------- Define features & targets ----------------------#

FEATURE_COLS = [
    "rsi",
    "rsi_ma",
    "stoch_k",
    "stoch_d",
    "macd",
    "macd_signal",
    "macd_hist",
    "vwma",
    "volume",
    "atr",
    "roc",
    "mom",
    "cci",
    "cci_ma",
    "conv_line",
    "base_line",
    "span_a",
    "span_b",
]

existing_features = [c for c in FEATURE_COLS if c in closed.columns]
missing_feat = [c for c in FEATURE_COLS if c not in closed.columns]
if missing_feat:
    print("⚠️ Missing feature columns in merged data:", missing_feat)
print("\nUsing feature columns:")
print(existing_features)

closed_feat = closed.dropna(subset=existing_features).copy()

if "runup_pct" not in closed_feat.columns or "drawdown_pct" not in closed_feat.columns:
    raise ValueError("runup_pct / drawdown_pct not found in trades data.")

#---------------------- Bucketing functions ----------------------#

def bucket_runup(x):
    if x < 2:
        return 0
    elif x < 5:
        return 1
    elif x < 10:
        return 2
    else:
        return 3

def bucket_drawdown(x):
    if x > -1:
        return 0
    elif x > -3:
        return 1
    else:
        return 2

closed_feat["runup_class"] = closed_feat["runup_pct"].astype(float).apply(bucket_runup)
closed_feat["drawdown_class"] = closed_feat["drawdown_pct"].astype(float).apply(bucket_drawdown)

print("\nRun-up class distribution:")
print(closed_feat["runup_class"].value_counts().sort_index())
print("\nDrawdown class distribution:")
print(closed_feat["drawdown_class"].value_counts().sort_index())

#---------------------- Build X, y ----------------------#

X = closed_feat[existing_features].astype(float).values
y_run = closed_feat["runup_class"].values
y_dd  = closed_feat["drawdown_class"].values

print("\nML dataset shape:", X.shape)

n = len(X)
split_idx = int(n * 0.7)
X_train, X_test = X[:split_idx], X[split_idx:]
y_run_train, y_run_test = y_run[:split_idx], y_run[split_idx:]
y_dd_train,  y_dd_test  = y_dd[:split_idx],  y_dd[split_idx:]

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")


Merged trades + features sample:
   trade_id                entry_time  entry_price           exit_time  \
0         1 2024-01-03 08:30:00+00:00      42478.0 2024-01-04 08:30:00   
1         2 2024-01-05 13:00:00+00:00      43603.6 2024-01-05 19:00:00   
2         3 2024-01-07 22:45:00+00:00      43477.7 2024-01-09 11:30:00   
3         4 2024-01-12 04:00:00+00:00      45862.2 2024-01-15 14:30:00   
4         5 2024-01-17 13:00:00+00:00      42375.5 2024-01-18 03:15:00   
5         6 2024-01-18 13:00:00+00:00      41813.9 2024-01-19 03:15:00   
6         7 2024-01-19 13:00:00+00:00      41239.2 2024-01-20 15:15:00   
7         8 2024-01-21 12:15:00+00:00      41645.0 2024-01-24 03:15:00   
8         9 2024-01-24 15:15:00+00:00      39924.7 2024-01-24 20:30:00   
9        10 2024-01-29 10:45:00+00:00      42082.6 2024-01-29 12:15:00   

   exit_price  net_pnl_pct  runup_pct  drawdown_pct                      time  \
0     43242.1         1.80       2.63         -1.31 2024-01-03 08:15:00

In [24]:
#===============================================================================
# CELL 3: Regression + Binary models for Run-up & Drawdown
#===============================================================================

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    classification_report,
)
import numpy as np

# نتأكد إننا ماشين بنفس الأوردر الزمني المستخدم في closed_feat
X_all = closed_feat[existing_features].astype(float).values

#---------------------- Targets كـ أرقام (Regression) ----------------------#

y_run_cont = closed_feat["runup_pct"].astype(float).values       # أقصى ربح %
y_dd_cont  = closed_feat["drawdown_pct"].astype(float).values    # أقصى درو داون %

n = len(X_all)
split_idx = int(n * 0.7)

X_train, X_test = X_all[:split_idx], X_all[split_idx:]
y_run_train, y_run_test = y_run_cont[:split_idx], y_run_cont[split_idx:]
y_dd_train,  y_dd_test  = y_dd_cont[:split_idx],  y_dd_cont[split_idx:]

#---------------------- RandomForest Regression models ----------------------#

run_reg = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
)

dd_reg = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
)

run_reg.fit(X_train, y_run_train)
dd_reg.fit(X_train, y_dd_train)

# تقييم الموديلين
y_run_pred = run_reg.predict(X_test)
y_dd_pred  = dd_reg.predict(X_test)

print("=== Run-up Regression (Test) ===")
print(f"MAE     : {mean_absolute_error(y_run_test, y_run_pred):.3f} %")
print(f"RMSE    : {np.sqrt(mean_squared_error(y_run_test, y_run_pred)):.3f} %")
print(f"R^2     : {r2_score(y_run_test, y_run_pred):.3f}\n")

print("=== Drawdown Regression (Test) ===")
print(f"MAE     : {mean_absolute_error(y_dd_test, y_dd_pred):.3f} %")
print(f"RMSE    : {np.sqrt(mean_squared_error(y_dd_test, y_dd_pred)):.3f} %")
print(f"R^2     : {r2_score(y_dd_test, y_dd_pred):.3f}\n")

# نحتفظ بالموديلز عشان نستخدمهم في prediction بعدين
runup_reg_model    = run_reg
drawdown_reg_model = dd_reg

#---------------------- Binary targets (تحويل لأكتر / أقل من threshold) ----------------------#
# Run-up:
#   0 → أقصى ربح أقل من 3%
#   1 → أقصى ربح أكبر أو يساوي 3%
#
# Drawdown:
#   0 → أقصى درو داون "مش مرعب" (أعلى من -5%)
#   1 → أقصى درو داون عنيف (أقل أو يساوي -5%)

def bucket_runup_binary(x, thr=3.0):
    return 1 if x >= thr else 0

def bucket_dd_binary(x, thr=-5.0):
    return 1 if x <= thr else 0

y_run_bin = closed_feat["runup_pct"].astype(float).apply(bucket_runup_binary).values
y_dd_bin  = closed_feat["drawdown_pct"].astype(float).apply(bucket_dd_binary).values

print("Run-up binary class balance (0:<3%, 1:>=3%):")
unique, counts = np.unique(y_run_bin, return_counts=True)
print(dict(zip(unique, counts)))

print("\nDrawdown binary class balance (0:>-5%, 1:<=-5%):")
unique, counts = np.unique(y_dd_bin, return_counts=True)
print(dict(zip(unique, counts)), "\n")

# نفس الـ split الزمني
y_run_bin_train, y_run_bin_test = y_run_bin[:split_idx], y_run_bin[split_idx:]
y_dd_bin_train,  y_dd_bin_test  = y_dd_bin[:split_idx],  y_dd_bin[split_idx:]

#---------------------- RandomForest Binary Classifiers ----------------------#

run_bin = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1,
)

dd_bin = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1,
)

run_bin.fit(X_train, y_run_bin_train)
dd_bin.fit(X_train, y_dd_bin_train)

# تقييم
print("=== Run-up Binary Model (>= 3% ?) ===")
y_run_bin_pred = run_bin.predict(X_test)
print(classification_report(y_run_bin_test, y_run_bin_pred, digits=3))

print("=== Drawdown Binary Model (<= -5% ?) ===")
y_dd_bin_pred = dd_bin.predict(X_test)
print(classification_report(y_dd_bin_test, y_dd_bin_pred, digits=3))

# نخزنهم في variables عشان نستخدمهم في التريد المفتوحة
runup_bin_model    = run_bin
drawdown_bin_model = dd_bin

print("\n✅ Regression + Binary models trained and stored.")


=== Run-up Regression (Test) ===
MAE     : 0.883 %
RMSE    : 1.119 %
R^2     : -0.369

=== Drawdown Regression (Test) ===
MAE     : 2.131 %
RMSE    : 2.523 %
R^2     : -0.975

Run-up binary class balance (0:<3%, 1:>=3%):
{np.int64(0): np.int64(193), np.int64(1): np.int64(25)}

Drawdown binary class balance (0:>-5%, 1:<=-5%):
{np.int64(0): np.int64(195), np.int64(1): np.int64(23)} 

=== Run-up Binary Model (>= 3% ?) ===
              precision    recall  f1-score   support

           0      0.921     0.951     0.935        61
           1      0.000     0.000     0.000         5

    accuracy                          0.879        66
   macro avg      0.460     0.475     0.468        66
weighted avg      0.851     0.879     0.865        66

=== Drawdown Binary Model (<= -5% ?) ===
              precision    recall  f1-score   support

           0      0.952     0.937     0.944        63
           1      0.000     0.000     0.000         3

    accuracy                          0.894  

In [25]:
#===============================================================================
# CELL 4: Prediction for LAST OPEN TRADE using Regression + Binary models
#===============================================================================

if len(open_for_pred) == 0:
    print("⚠️ No open trades available for prediction.")
else:
    # ناخد آخر صفقة مفتوحة زمنياً
    last_open = open_for_pred.sort_values("entry_time").iloc[-1]

    # نجهز الـ features
    feat_row = last_open[existing_features]

    if feat_row.isna().any():
        print("⚠️ Cannot predict: open trade row still has NaNs in feature columns.")
        print("NaN features:", feat_row[feat_row.isna()])
    else:
        X_live = feat_row.astype(float).values.reshape(1, -1)

        trade_id   = last_open.get("trade_id", None)
        entry_time = last_open["entry_time"]
        entry_price = last_open["entry_price"]
        cur_pnl = last_open.get("net_pnl_pct", np.nan)

        print("============================================")
        print("📌 LAST OPEN TRADE - FULL PREDICTION VIEW")
        print("============================================")
        print(f"Trade ID   : {trade_id}")
        print(f"Entry time : {entry_time}")
        print(f"Entry price: {entry_price:,.2f}")
        if not np.isnan(cur_pnl):
            print(f"Current P&L: {cur_pnl:+.2f}% (floating)")
        print("--------------------------------------------\n")

        #---------------------- 1) Regression outputs ----------------------#
        runup_pred_cont   = runup_reg_model.predict(X_live)[0]
        drawdown_pred_cont = drawdown_reg_model.predict(X_live)[0]

        print("🔮 Regression estimates (continuous):")
        print(f"  • Expected MAX run-up   ≈ {runup_pred_cont:+.2f}%")
        print(f"  • Expected MAX drawdown ≈ {drawdown_pred_cont:+.2f}%")
        print("    (interpretation: متوسط ما الموديل شايفه كأقصى ربح/خسارة في أحسن/أسوأ نقطة خلال عمر التريد)\n")

        #---------------------- 2) Binary probabilities ----------------------#
        # Run-up: 1 إذا run-up >= 3%
        runup_proba = runup_bin_model.predict_proba(X_live)[0]
        # Drawdown: 1 إذا drawdown <= -5%
        drawdown_proba = drawdown_bin_model.predict_proba(X_live)[0]

        print("🔮 Run-up >= 3% ? (binary model)")
        print(f"  • P(run-up < 3%)  = {runup_proba[0]*100:5.1f}%")
        print(f"  • P(run-up ≥ 3%)  = {runup_proba[1]*100:5.1f}%\n")

        print("🔮 Drawdown ≤ -5% ? (binary model)")
        print(f"  • P(drawdown > -5%) = {drawdown_proba[0]*100:5.1f}%")
        print(f"  • P(drawdown ≤ -5%) = {drawdown_proba[1]*100:5.1f}%")
        print("\n(ده كله للمساعدة في القرار، مش توصية مالية مباشرة 👀)")


📌 LAST OPEN TRADE - FULL PREDICTION VIEW
Trade ID   : 220
Entry time : 2025-11-11 09:15:00+00:00
Entry price: 104,340.00
Current P&L: -9.02% (floating)
--------------------------------------------

🔮 Regression estimates (continuous):
  • Expected MAX run-up   ≈ +1.84%
  • Expected MAX drawdown ≈ -2.83%
    (interpretation: متوسط ما الموديل شايفه كأقصى ربح/خسارة في أحسن/أسوأ نقطة خلال عمر التريد)

🔮 Run-up >= 3% ? (binary model)
  • P(run-up < 3%)  =  75.3%
  • P(run-up ≥ 3%)  =  24.7%

🔮 Drawdown ≤ -5% ? (binary model)
  • P(drawdown > -5%) =  76.8%
  • P(drawdown ≤ -5%) =  23.2%

(ده كله للمساعدة في القرار، مش توصية مالية مباشرة 👀)


In [26]:
#===============================================================================
# CELL 4: (Re)train models if needed + Hypothetical NEW ENTRY NOW
#===============================================================================
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report
import numpy as np

#---------------------------------------
# 1) لو الموديلات مش موجودة -> ندرّبها
#---------------------------------------
need_train = (
    "run_rf_reg" not in globals()
    or "dd_rf_reg" not in globals()
    or "run_clf_bin" not in globals()
    or "dd_clf_bin" not in globals()
)

if need_train:
    print("⚙️ Models not found in memory -> re-training quickly...\n")

    # X, y من الداتا اللي جهزناها في Cell 2
    X_all = closed_feat[existing_features].astype(float).values

    # Regression targets (continuous)
    y_run_reg = closed_feat["runup_pct"].astype(float).values      # max run-up %
    y_dd_reg  = closed_feat["drawdown_pct"].astype(float).values   # max drawdown %

    # Binary targets
    # run-up: 1 لو >= 3%, 0 لو < 3%
    y_run_bin = (closed_feat["runup_pct"].astype(float) >= 3.0).astype(int).values
    # drawdown: 1 لو <= -5%, 0 لو > -5%
    y_dd_bin  = (closed_feat["drawdown_pct"].astype(float) <= -5.0).astype(int).values

    # time-based split 70% / 30%
    n = len(X_all)
    split_idx = int(n * 0.7)

    X_train, X_test = X_all[:split_idx], X_all[split_idx:]
    y_run_reg_train, y_run_reg_test = y_run_reg[:split_idx], y_run_reg[split_idx:]
    y_dd_reg_train,  y_dd_reg_test  = y_dd_reg[:split_idx],  y_dd_reg[split_idx:]

    y_run_bin_train, y_run_bin_test = y_run_bin[:split_idx], y_run_bin[split_idx:]
    y_dd_bin_train,  y_dd_bin_test  = y_dd_bin[:split_idx],  y_dd_bin[split_idx:]

    # --- Regressors ---
    run_rf_reg = RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
    )
    dd_rf_reg = RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
    )

    run_rf_reg.fit(X_train, y_run_reg_train)
    dd_rf_reg.fit(X_train, y_dd_reg_train)

    # --- Binary classifiers ---
    run_clf_bin = RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=5,
        random_state=42,
        class_weight="balanced_subsample",
        n_jobs=-1,
    )
    dd_clf_bin = RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=5,
        random_state=42,
        class_weight="balanced_subsample",
        n_jobs=-1,
    )

    run_clf_bin.fit(X_train, y_run_bin_train)
    dd_clf_bin.fit(X_train, y_dd_bin_train)

    # (اختياري) تطبع شوية أرقام تأكيد
    y_run_pred_test = run_rf_reg.predict(X_test)
    y_dd_pred_test  = dd_rf_reg.predict(X_test)

    print("=== Quick check: Run-up Regression (Test) ===")
    print("MAE  :", f"{mean_absolute_error(y_run_reg_test, y_run_pred_test):.3f}", "%")
    print("RMSE :", f"{mean_squared_error(y_run_reg_test, y_run_pred_test, squared=False):.3f}", "%")
    print("R^2  :", f"{r2_score(y_run_reg_test, y_run_pred_test):.3f}")
    print("\n=== Quick check: Drawdown Regression (Test) ===")
    print("MAE  :", f"{mean_absolute_error(y_dd_reg_test, y_dd_pred_test):.3f}", "%")
    print("RMSE :", f"{mean_squared_error(y_dd_reg_test, y_dd_pred_test, squared=False):.3f}", "%")
    print("R^2  :", f"{r2_score(y_dd_reg_test, y_dd_pred_test):.3f}")

    print("\n(Models are ready now.)\n")

#---------------------------------------
# 2) Hypothetical NEW ENTRY from last bar
#---------------------------------------

# آخر بار من الأسعار
last_bar = prices.sort_values("time").iloc[-1].copy()
t_now = last_bar["time"]
price_now = last_bar["close"]

print("============================================")
print("📌 HYPOTHETICAL NEW LONG ENTRY *NOW*")
print("============================================")
print(f"Time        : {t_now}")
print(f"Entry price : {price_now:,.2f}")

# medians لتعويض أي NaNs
feature_medians = closed_feat[existing_features].median()

# نبني feature vector للبار الحالي
x_now_series = last_bar[existing_features].copy()
x_now_series = x_now_series.fillna(feature_medians)
x_now = x_now_series.astype(float).values.reshape(1, -1)

# --- Regression predictions ---
runup_pred_now = run_rf_reg.predict(x_now)[0]
dd_pred_now    = dd_rf_reg.predict(x_now)[0]

# --- Binary predictions ---
runup_proba_now = run_clf_bin.predict_proba(x_now)[0]
dd_proba_now    = dd_clf_bin.predict_proba(x_now)[0]

p_run_lt3 = 100 * (1 - runup_proba_now[1])
p_run_ge3 = 100 * runup_proba_now[1]
p_dd_gt5  = 100 * (1 - dd_proba_now[1])
p_dd_le5  = 100 * dd_proba_now[1]

print("\n🔮 Regression estimates for THIS NEW entry:")
print(f"  • Expected MAX run-up   ≈ {runup_pred_now:+.2f}%")
print(f"  • Expected MAX drawdown ≈ {dd_pred_now:+.2f}%")

print("\n🔮 Run-up ≥ 3% ? (binary model)")
print(f"  • P(run-up < 3%)  = {p_run_lt3:5.1f}%")
print(f"  • P(run-up ≥ 3%)  = {p_run_ge3:5.1f}%")

print("\n🔮 Drawdown ≤ -5% ? (binary model)")
print(f"  • P(drawdown > -5%) = {p_dd_gt5:5.1f}%")
print(f"  • P(drawdown ≤ -5%) = {p_dd_le5:5.1f}%")

print("\n(ده بيقول لك: لو دخلت لونج دلوقتي من آخر شمعة،")
print(" الموديل شايف تقريبًا أقصى ربح/درو داون هيبقوا قد إيه،")
print(" وإحتمال توصل لـ +3% ربح أو تضرب -5% درو داون قد إيه 👀)")


📌 HYPOTHETICAL NEW LONG ENTRY *NOW*
Time        : 2025-11-15 04:00:00-05:00
Entry price : 95,748.40

🔮 Regression estimates for THIS NEW entry:
  • Expected MAX run-up   ≈ +2.12%
  • Expected MAX drawdown ≈ -2.79%

🔮 Run-up ≥ 3% ? (binary model)
  • P(run-up < 3%)  =  67.0%
  • P(run-up ≥ 3%)  =  33.0%

🔮 Drawdown ≤ -5% ? (binary model)
  • P(drawdown > -5%) =  74.2%
  • P(drawdown ≤ -5%) =  25.8%

(ده بيقول لك: لو دخلت لونج دلوقتي من آخر شمعة،
 الموديل شايف تقريبًا أقصى ربح/درو داون هيبقوا قد إيه،
 وإحتمال توصل لـ +3% ربح أو تضرب -5% درو داون قد إيه 👀)
